In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [0]:
%cd "../../content/drive/My Drive/Colab Notebooks/EmoteNN"

/content/drive/My Drive/Colab Notebooks/EmoteNN


In [0]:
import os
import numpy as np
from emotenn import constants as ct, load_utils as lu, train_utils as tu
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import callbacks
import tensorflow as tf

In [0]:
X, Y = lu.load_dataset(ct.FER2013)
img_count, w, h, _ = X.shape

X_train, X_valid, X_test, Y_train, Y_valid, Y_test = tu.split_dataset(X, Y, test_size=0.15, valid_size=0.2)
print(f'Train data dimensions: {X_train.shape}')
print(f'Validation data dimensions: {X_valid.shape}')
print(f'Test data dimensions: {X_test.shape}')
X_train = np.array(X_train, dtype=np.float32)
Y_train = np.array(Y_train, dtype=np.float32)
X_test = np.array(X_test, dtype=np.float32)
Y_test = np.array(Y_test, dtype=np.float32)
X_valid = np.array(X_valid, dtype=np.float32)
Y_valid = np.array(Y_valid, dtype=np.float32)


Train data dimensions: (23326, 48, 48, 1)
Validation data dimensions: (7177, 48, 48, 1)
Test data dimensions: (5384, 48, 48, 1)


In [0]:
from models import model as mdl

features_count = 64
labels_count = 7

In [0]:
batch_size = 512
epochs = 10
model_name = 'model'

# TPU detection  
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
except ValueError:
  tpu = None

# TPUStrategy for distributed training
if tpu:
  print("TPUStrategy: ")
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.experimental.TPUStrategy(tpu)
  with strategy.scope():
    model = mdl.build_model(features_count, labels_count, w, h)
    model.compile(loss=categorical_crossentropy,
                  optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7),
                  metrics=['accuracy'])
    steps_per_epoch = 45
else: # default strategy that works on CPU and single GPU
  model = mdl.build_model(features_count, labels_count, w, h)
  model.compile(loss=categorical_crossentropy,
                optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7),
                metrics=['accuracy'])
  steps_per_epoch = None

callbacks = tu.get_callbacks(model_name)
train_history = model.fit(X_train,
                          Y_train,
                          batch_size=batch_size,
                          epochs=epochs,
                          verbose=1,
                          # validation_data=(X_valid, Y_valid),
                          shuffle=True, 
                          steps_per_epoch=steps_per_epoch)

TPUStrategy: 
INFO:tensorflow:Initializing the TPU system: 10.31.14.202:8470
INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Querying Tensorflow master (grpc://10.31.14.202:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 14135677787981631722)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 110405886981013896)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 13301472445295855822)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 9477574458504106835)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:

In [0]:
score = model.evaluate(X_test, Y_test)
print("Dataset {}: {:.2f}%".format(model.metrics_names[1], score[1] * 100))

169/169 [==============================] - 8s 45ms/step
Dataset acc: 42.22%


In [0]:
tu.save_results(model, train_history, model_name)